In [1]:
import resnet50,sys,os
import numpy as np
import tensorflow.compat.v1 as tf
from keras import backend as K
import tables
tf.disable_eager_execution()

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
custom_weights_dir = os.path.expanduser("/home/david/Storage/dataset_Tom/weights-floatingpoint-224x224-fixval-best")
saved_model_dir = os.path.expanduser("/home/david/Storage/dataset_Tom/weights-floatingpoint-224x224-fixval-best/")

In [3]:
def normalize_and_rgb(images): 
    #normalize image to 0-255 per image.
    image_sum = 1/np.sum(np.sum(images,axis=1),axis=-1)
    given_axis = 0
    # Create an array which would be used to reshape 1D array, b to have 
    # singleton dimensions except for the given axis where we would put -1 
    # signifying to use the entire length of elements along that axis  
    dim_array = np.ones((1,images.ndim),int).ravel()
    dim_array[given_axis] = -1
    # Reshape b with dim_array and perform elementwise multiplication with 
    # broadcasting along the singleton dimensions for the final output
    image_sum_reshaped = image_sum.reshape(dim_array)
    images = images*image_sum_reshaped*255

    # make it rgb by duplicating 3 channels.
    images = np.stack([images, images, images],axis=-1)
    
    return images


In [4]:

import glob
import random

datadir = "/home/david/Storage/dataset_Tom/img224_all/converted/rotation_224_v1/"
num_train = 100  # Limit the number of images used in training to shorten epoch time

train_files = glob.glob(os.path.join(datadir, 'train_file_*'))
train_files = random.choice(train_files)  # Choose one of the training files to use (for now)

# Open the chosen file and extract the dataset
f = tables.open_file(train_files, 'r')
a = np.array(f.root.img_pt) # Images
b = np.array(f.root.label) # Labels
# Randomly shuffle label and images, keep the indexing
c = np.c_[a.reshape(len(a), -1), b.reshape(len(b), -1)]
np.random.shuffle(c)
train_images = c[:, :a.size//len(a)].reshape(a.shape)
train_labels = c[:, a.size//len(a):].reshape(b.shape)

# Limit the data set to make the notebook execute quickly.
train_images = train_images[:num_train]
train_labels = train_labels[:num_train]

train_images = normalize_and_rgb(train_images)

In [5]:
num_test = 100

test_files = glob.glob(os.path.join(datadir, '/home/david/Storage/dataset_Tom/img224_all/converted/rotation_224_v1/test_file_*'))
test_files = random.choice(test_files)  # Choose one of the training files to use (for now)

# Open the chosen file and extract the dataset
f = tables.open_file(test_files, 'r')
a = np.array(f.root.img_pt) # Images
b = np.array(f.root.label) # Labels
# Randomly shuffle label and images, keep the indexing
c = np.c_[a.reshape(len(a), -1), b.reshape(len(b), -1)]
np.random.shuffle(c)
test_images = c[:, :a.size//len(a)].reshape(a.shape)
test_labels = c[:, a.size//len(a):].reshape(b.shape)

# Limit the data set to make the notebook execute quickly.
test_images = test_images[:num_test]
test_labels = test_labels[:num_test]

test_images = normalize_and_rgb(test_images)

In [6]:
def preprocess_images():
    # Create a placeholder for our incoming images
    in_images = tf.placeholder(tf.float32)
    in_height = 64
    in_width = 64
    in_images.set_shape([None, in_height, in_width, 3])
    
    # Resize those images to fit our featurizer
    out_width = 224
    out_height = 224
    image_tensors = tf.image.resize_images(in_images, [out_height,out_width])
    image_tensors = tf.to_float(image_tensors)
    
    return in_images, image_tensors

In [11]:
def construct_classifier(in_tensor):
    from keras.layers import Dropout, Dense, Flatten
    K.set_session(tf.get_default_session())
    
    FC_SIZE = 1024
    NUM_CLASSES = 2

    x = Dropout(0.2, input_shape=(1, 1, 2048,))(in_tensor)
    x = Dense(FC_SIZE, activation='relu', input_dim=(1, 1, 2048,))(x)
    x = Flatten()(x)
    preds = Dense(NUM_CLASSES, activation='softmax', input_dim=FC_SIZE, name='classifier_output')(x)
    return preds

def construct_model(quantized, starting_weights_directory = None):
    
    # Convert images to 3D tensors [width,height,channel]
    in_images, image_tensors = preprocess_images()

    # Construct featurizer using quantized or unquantized ResNet50 model
    if not quantized:
        featurizer = resnet50.ResNet50(data_format())
    else:
        #featurizer = QuantizedResnet50(saved_model_dir, custom_weights_directory = starting_weights_directory)
        print("quantize mode disabled")
    features = featurizer
    # Construct classifier
    preds = construct_classifier(features(in_images, training=True))

    # Initialize weights
    sess = tf.get_default_session()
    tf.global_variables_initializer().run()

    featurizer.restore_weights(sess)

    return in_images, image_tensors, features, preds, featurizer

In [12]:
def data_format():
    return 'channels_last' if tf.test.is_gpu_available() else 'channels_first'

def train_model(preds, in_images, train_images, train_labels, train_epoch = 10):
    """ training model """
    from keras.objectives import binary_crossentropy
    from keras.metrics import categorical_accuracy
    from tqdm import tqdm
    
    learning_rate = 0.001 
        
    # Specify the loss function
    in_labels = tf.placeholder(tf.float32, shape=(None, 2))   
    cross_entropy = tf.reduce_mean(binary_crossentropy(in_labels, preds))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)
    
    accuracy = tf.reduce_mean(categorical_accuracy(in_labels, preds))
    auc = tf.metrics.auc(tf.cast(in_labels, tf.bool), preds)
    
    def chunks(a, b, n):
        """Yield successive n-sized chunks from a and b."""
        for i in range(0, num_train, n):
            yield a[i:i + n], b[i:i + n]

    chunk_size = 16
    chunk_num = len(train_labels) / chunk_size

    sess = tf.get_default_session()
    sess.run(tf.group(tf.global_variables_initializer(), tf.local_variables_initializer()))
    
    loss_over_epoch = []
    accuracy_over_epoch = []
    auc_over_epoch = []
    
    for epoch in range(train_epoch):
        avg_loss = 0
        avg_accuracy = 0
        avg_auc = 0
        for img_chunk, label_chunk in tqdm(chunks(train_images, train_labels, chunk_size)):
            _, loss = sess.run([optimizer, cross_entropy],
                                feed_dict={in_images: img_chunk,
                                           in_labels: label_chunk,
                                           K.learning_phase(): 1})
            avg_loss += loss / chunk_num
            accuracy_result, auc_result = sess.run([accuracy, auc],
                                     feed_dict={in_images: test_images,
                                                in_labels: test_labels,
                                                K.learning_phase(): 0})
            avg_accuracy += accuracy_result / chunk_num
            avg_auc += auc_result[1] / chunk_num
        print("Epoch:", (epoch + 1), "loss = ", "{:.3f}".format(avg_loss))
        print("Accuracy:", "{:.3f}".format(avg_accuracy), ", Area under ROC curve:", "{:.3f}".format(avg_auc))
            
        loss_over_epoch.append(avg_loss)
        accuracy_over_epoch.append(avg_accuracy)
        auc_over_epoch.append(avg_auc)
            
        # Reach desired performance
        if (avg_loss < 0.001):
            break
            
    return loss_over_epoch, accuracy_over_epoch, auc_over_epoch

In [13]:
def test_model(preds, in_images, test_images, test_labels):
    """Test the model"""
    from keras.metrics import categorical_accuracy

    in_labels = tf.placeholder(tf.float32, shape=(None, 2))
    accuracy = tf.reduce_mean(categorical_accuracy(in_labels, preds))
    auc = tf.metrics.auc(tf.cast(in_labels, tf.bool), preds)
    
    sess.run(tf.group(tf.global_variables_initializer(), tf.local_variables_initializer()))
    accuracy, auc = sess.run([accuracy, auc],
                    feed_dict={in_images: test_images,
                               in_labels: test_labels,
                               K.learning_phase(): 0})
    
    return accuracy, auc

In [14]:
# Launch the training
tf.reset_default_graph()
sess = tf.Session(graph=tf.get_default_graph())

num_epoch_train = 10

with sess.as_default():
    in_images, image_tensors, features, preds, featurizer = construct_model(quantized=False)
    loss_over_epoch, accuracy_over_epoch, auc_over_epoch = \
        train_model(preds, in_images, train_images, train_labels, train_epoch=num_epoch_train)    
    accuracy, auc = test_model(preds, in_images, test_images, test_labels)  
    # print("Accuracy:", accuracy, ", Area under ROC curve:", auc)
    featurizer.save_weights(saved_model_dir + "/rn50/1.1.3/resnet50_bw", tf.get_default_session())

ValueError: in converted code:

    /home/david/workplace/Top-tagging-keras/resnet50.py:357 call *
        x = self.avg_pool(x)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer.py:634 __call__
        outputs = call_fn(inputs, *args, **kwargs)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/layers/pooling.py:248 call
        data_format=conv_utils.convert_data_format(self.data_format, 4))
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_ops.py:3528 avg_pool
        name=name)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gen_nn_ops.py:95 avg_pool
        data_format=data_format, name=name)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/op_def_library.py:788 _apply_op_helper
        op_def=op_def)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/util/deprecation.py:507 new_func
        return func(*args, **kwargs)
    /usr/local/lib/python3.6/dist-packages/ten